In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
alert_data = pd.read_csv(
    "dataset/alert.csv",
    index_col=0, 
    encoding="utf-8"
)
alert_ident = ['src_ip', 'dst_ip', 'src_port', 'dst_port']
alert_data.shape

(489850, 15)

In [3]:
netflow_feats = pd.read_csv("dataset/netflow/netflow-feats.csv").columns
netflow_list = os.listdir("dataset/netflow/flow")
np.random.shuffle(netflow_list)

In [4]:
selected_features = [
    'src_port', 
    'dst_port',
    
    'ip_proto_pkt_cnt',
    'ip_proto_bytes_cnt',
    
    'tcp_proto_pkt_cnt',
    'tcp_proto_syn_cnt',
    'tcp_proto_fin_cnt', 
    'tcp_proto_psh_cnt',
    'tcp_proto_rst_cnt',
    "tcp_proto_synack_cnt",
    "tcp_proto_ack_cnt",
    "tcp_proto_bytes_cnt", 
    
    'udp_proto_pkt_cnt',
    'udp_proto_bytes_cnt', 
    
    'icmp_proto_pkt_cnt',
    'icmp_proto_echo_reply_cnt',
    "icmp_proto_bytes_cnt",
    "icmp_proto_echo_request_cnt",
]

times_features = [
    'first_alert_time_sec',
    'last_alert_time_sec',
    'first_pkt_time_sec',
    'last_pkt_time_sec'
]

In [5]:
balanced_batches = []
batch_size = 20 # files
batch_start, batch_end = 0, 2 

In [6]:
%%time
for batch_count in range(batch_start, batch_end):
    netflow_data = pd.concat(
        [
            pd.concat(
                [
                    pd.read_csv(
                        f"dataset/netflow/flow/{filename}", 
                        names=netflow_feats,  
                        encoding="cp1251"
                    ) for filename in netflow_list[s:s+5]
                ],
                ignore_index=True
            ).merge(
                alert_data, 
                how='left', 
                on=alert_ident,
                indicator=True
            ) [selected_features + times_features + ['_merge']] 
            for s in range(20 * batch_count, 20 * (batch_count + 1), 5)
        ],
        ignore_index=True
    )
    netflow_data['is_alert'] = netflow_data['_merge'].replace(
        {
            'both': 1, 'left_only': 0, 'right_only': 0
        },
    )
    netflow_data = netflow_data[
        (
            (
                netflow_data['is_alert'] == 1
            ) & (
                netflow_data['first_alert_time_sec'] >= netflow_data['first_pkt_time_sec']
            ) & (
                netflow_data['first_alert_time_sec'] <= netflow_data['last_pkt_time_sec' ]
            ) & (
                netflow_data['last_alert_time_sec' ] <= netflow_data['last_pkt_time_sec' ]
            )
        ) | (
            netflow_data['is_alert'] == 0
        )
    ][selected_features + ['is_alert']].drop_duplicates()

    # random sampling
    alert_index = netflow_data[netflow_data.is_alert == 1].index

    balanced_batches.append(
        netflow_data.loc[
            np.concatenate(
                (
                    np.random.choice(
                        netflow_data[netflow_data.is_alert == 0].index, 
                        size=alert_index.shape[0] * 49,
                        replace=False
                    ), 
                    alert_index
                ), 
                axis=0
            )
        ]
    )

CPU times: user 46.1 s, sys: 10.8 s, total: 56.8 s
Wall time: 1min 9s


In [7]:
X = pd.concat(
    balanced_batches,
    sort=False,
    ignore_index=True
).drop_duplicates()

y = X['is_alert']
X = X[features]
X.shape

NameError: name 'features' is not defined